Learnings:

Given a custom type defined with `weave.type()` (& optionally a series of additional ops):
* You can run `weave.use` on the tail node and sucessfully get a response
* Running `weave.show` is broken with `TypeError: Object of type CUSTOM_TYPE is not JSON serializable`.
    * This is because `def show` needs to convert the requested object to a JSON document describing the object
        to the iframe (via URL parameter), which in turn needs to send the same (or modified) JSON back to a python
        engine. 
    * The first, not good solution, is to call `weave.save` on the object before passing it to the ops. This converts
        the object to a refrence, which can in fact be serialized. While this makes it work, it is not a reasonable
        ask of users.
    * This is becuase:
        a. when the ConstNode is serialized, it blindly assigns it's value to the value - which in these cases, are not primitive in themselves
        b. assuming we recurssively transform into a dictionary, we will reach some types that are not to_jsonable (like pytorch model)
        c. in those cases, we need to basically convert to a reference (similar to weave.save).
    * My problem is figuring out where to do this - in mappers, in the to_json of ConstNode? How does this relate to the instance_to/from_dict of the types? It feels like the later (which could be generallized in the ObjectType type). 
    * Followup, i want it to look like you are calling the constructor (like MyType(4, 5))
    
Potential Plan:
* Make @weave.type() constructors somehow a weave op that returns an instance of this type. This will preserve the UI of TypeName(params)
* Const nodes should attempt to save/serialize their value before writing it if it is of a special type.
    
```
from weave.show import _show_params

a = _show_params(addMagnitude(TypeExampleOne(45)))
a['weave_node'].to_json()

{'nodeType': 'output',
 'type': 'int',
 'fromOp': {'name': 'op-addMagnitude',
  'inputs': {'a': {'nodeType': 'const',
    'type': {'type': 'const',
     'valType': {'type': 'TypeExampleOne'},
     'val': TypeExampleOne(attr=45)},
    'val': TypeExampleOne(attr=45)}}}}
```

In [5]:
# !rm -rf /tmp/local-artifacts

In [1]:
IS_CI = False

In [2]:
import weave
import random

# Randomness allows us to avoid cache hits
def rand_int():
    return int(random.random() * 1e10)

def run_test(label, test_case):
    print(f"Test: Runtime, {label}")
    print(f"{weave.use(test_case())=}")
    weave.show(test_case())

    print(f"Test: Local Save, {label}")
    print(f"{weave.use(weave.save(test_case()))=}")
    weave.show(weave.save(test_case()))
    
    # CI does not support publishing
    if not IS_CI:
        print(f"Test: Remote Publish, {label}")
        print(f"{weave.use(weave.publish(test_case()))=}")
        weave.show(weave.publish(test_case()))

In [4]:
run_test("Primitive Type", rand_int)

Test: Runtime, Primitive Type
weave.use(test_case())=2245773010


Test: Local Save, Primitive Type
weave.use(weave.save(test_case()))=2695355217


Test: Remote Publish, Primitive Type
weave.use(weave.publish(test_case()))=6866243853


In [3]:
@weave.op(render_info={'type': 'function'})
def make_number(val: int) -> int:
    return val

In [4]:
def make_number_op():
    return make_number(rand_int())
run_test("Op Returning Primitive Type", make_number_op)

Test: Runtime, Op Returning Primitive Type
weave.use(test_case())=6802277935


Test: Local Save, Op Returning Primitive Type
weave.use(weave.save(test_case()))=1834710707


Test: Remote Publish, Op Returning Primitive Type
weave.use(weave.publish(test_case()))=6348600454


In [ ]:
@weave.type()
class Fraction:
    numerator: int
    denominator: int

    @weave.op()
    def as_float(self) -> float:
        return self.numerator / self.denominator
    
    # UGG - this should be automatic!
    @weave.op()
    def get_numerator(self) -> int:
        return self.numerator
    
    # UGG - this should be automatic!
    @weave.op()
    def get_denominator(self) -> int:
        return self.denominator
    
@weave.op()
def show_fraction(frac: weave.Node[Fraction]) -> weave.panels.Group:
    return weave.panels.Group(
        items=[
            weave.panels.LabeledItem(label="numerator", item=frac.get_numerator()),
            weave.panels.LabeledItem(label="denominator", item=frac.get_denominator()),
            weave.panels.LabeledItem(label="fraction", item=frac.as_float()),
        ],
    ) 

def make_fraction():
    return Fraction(rand_int(), rand_int())

In [ ]:
# # broken cases:
# # weave.show(make_fraction().as_float()) # Broken
# # weave.show(weave.save(make_fraction().as_float())) # Broken

# # Custom testing
# import weave
# from weave.show import _show_params, make_refs
# # import json

# @weave.op()
# def make_number(val: int) -> int:
#     return val

# # obj = weave.save(make_fraction().as_float())
# # obj = weave.save(make_fraction().as_float())
# obj = weave.save(make_number(1))
# # obj = make_fraction()
# # params = _show_params(obj)
# ref = make_refs(obj)
# print(ref)
# # json.dumps(params["weave_node"].to_json())
# obj.from_op.name
# type(obj)

In [ ]:
run_test("Custom Type", make_fraction)

In [ ]:
def make_custom_chained():
    return make_fraction().as_float()

run_test("Custom Type with Chained Op", make_custom_chained)